<a href="https://colab.research.google.com/github/juanfranbrv/curso-langchain/blob/main/Cadena%20sencilla%20con%20LangGraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Configuración del entorno del cuaderno**



In [2]:
%%capture --no-stderr

# Importar la librería `userdata` de Google Colab.
# Esta librería se utiliza para acceder a datos de usuario almacenados de forma segura en el entorno de Colab.
from google.colab import userdata

# Obtener las claves API de diferentes servicios desde el almacenamiento seguro de Colab.
OPENAI_API_KEY=userdata.get('OPENAI_API_KEY')
# GROQ_API_KEY=userdata.get('GROQ_API_KEY')
# GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')
# HUGGINGFACEHUB_API_TOKEN=userdata.get('HUGGINGFACEHUB_API_TOKEN')

# El flag `-qU` instala en modo silencioso (`-q`) y actualiza las librerías si ya están instaladas (`-U`).
%pip install langchain -qU  # Instalar la librería principal de LangChain.
%pip install langgraph -qU

# Instalar las integraciones de LangChain con diferentes proveedores de LLMs.
%pip install langchain-openai -qU
# %pip install langchain-groq -qU
# %pip install langchain-google-genai -qU
# %pip install langchain-huggingface -qU

# Importar las clases necesarias de LangChain para crear plantillas de prompt.
# `ChatPromptTemplate` es la clase base para plantillas de chat.
from langchain.prompts import ChatPromptTemplate


# Importamos las clases necesarias para trabajar con cadenas
from langchain.chains import LLMChain

# Importar las clases para interactuar con los diferentes LLMs a través de LangChain.
from langchain_openai import ChatOpenAI
# from langchain_groq import ChatGroq
# from langchain_google_genai import ChatGoogleGenerativeAI
# from langchain_huggingface import HuggingFaceEndpoint

# Importamos la libreria para formatear mejor la salida
from IPython.display import Markdown, display

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.8/454.8 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 42.9 MB/s eta 0:00:00


Recordemos aqui que los modelos de lenguaje más modernos, como **gpt-3.5-turbo** y **gpt-4**, están diseñados para funcionar de manera óptima en un formato conversacional. Esto implica que, en lugar de enviar un único "prompt de texto" como entrada, podemos organizar la información en una secuencia de mensajes con roles específicos:

-   **system**: Mensaje del sistema (define el contexto o comportamiento del asistente).
    
-   **user**: Mensaje del usuario (preguntas o instrucciones del usuario).
    
-   **assistant**: Mensaje del asistente (respuestas generadas por el modelo).
    

LangChain admite varios tipos de mensajes, incluidos `HumanMessage`, `AIMessage`, `SystemMessage` y `ToolMessage`.

Estos representan un mensaje del usuario, del modelo de chat, para que el modelo de chat observe un comportamiento y de una llamada a una herramienta.


`**ChatPromptTemplate**` simplifica la creación de estos mensajes estructurados, evitando la necesidad de concatenarlos manualmente en una sola cadena de texto.

Creemos una lista de mensajes.


In [3]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    ("system", "Eres un experto en literatura fantastica y ciencia ficción"),
    ("human", "Proporciona un listado con los {numero} autores mas importantes de literatura fantastica en la actualidad"),
])


llm = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)

respuesta = llm.invoke(chat_prompt_template.format_messages(numero=5))

respuesta



AIMessage(content='Aquí tienes un listado con cinco de los autores más importantes de literatura fantástica en la actualidad:\n\n1. **Neil Gaiman**: Conocido por obras como "American Gods", "Coraline" y "The Ocean at the End of the Lane", Gaiman ha sido una figura central en la literatura fantástica contemporánea, combinando mitología, folclore y elementos de la cultura popular.\n\n2. **Patrick Rothfuss**: Su serie "The Kingkiller Chronicle", que incluye "The Name of the Wind" y "The Wise Man\'s Fear", ha sido aclamada por su prosa lírica y su profundo desarrollo de personajes, convirtiéndolo en un referente en la fantasía moderna.\n\n3. **N.K. Jemisin**: Ganadora de múltiples premios Hugo, Jemisin es conocida por su trilogía "The Broken Earth", que explora temas de opresión y poder en un mundo de fantasía complejo y original. Su trabajo ha sido fundamental para diversificar el género.\n\n4. **Brandon Sanderson**: Conocido por su habilidad para construir mundos y sistemas de magia comp

La respuesta del LLM es un objeto AIMessage.
Habitualmente la parte en la que estamos interesado es content



In [4]:
respuesta.content

'Aquí tienes un listado con cinco de los autores más importantes de literatura fantástica en la actualidad:\n\n1. **Neil Gaiman**: Conocido por obras como "American Gods", "Coraline" y "The Ocean at the End of the Lane", Gaiman ha sido una figura central en la literatura fantástica contemporánea, combinando mitología, folclore y elementos de la cultura popular.\n\n2. **Patrick Rothfuss**: Su serie "The Kingkiller Chronicle", que incluye "The Name of the Wind" y "The Wise Man\'s Fear", ha sido aclamada por su prosa lírica y su profundo desarrollo de personajes, convirtiéndolo en un referente en la fantasía moderna.\n\n3. **N.K. Jemisin**: Ganadora de múltiples premios Hugo, Jemisin es conocida por su trilogía "The Broken Earth", que explora temas de opresión y poder en un mundo de fantasía complejo y original. Su trabajo ha sido fundamental para diversificar el género.\n\n4. **Brandon Sanderson**: Conocido por su habilidad para construir mundos y sistemas de magia complejos, Sanderson h

Pero la respuesta contiene abundantes datos que en ocasiones son de utilidad

In [5]:
respuesta.response_metadata

{'token_usage': {'completion_tokens': 360,
  'prompt_tokens': 41,
  'total_tokens': 401,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-mini-2024-07-18',
 'system_fingerprint': 'fp_d02d531b47',
 'finish_reason': 'stop',
 'logprobs': None}

Las herramientas son útiles siempre que se desea que un modelo interactúe con sistemas externos.

Los sistemas externos (por ejemplo, las API) a menudo requieren un esquema de entrada o una carga útil particular, en lugar de lenguaje natural.

Cuando vinculamos una API, por ejemplo, como herramienta, le damos al modelo el conocimiento del esquema de entrada requerido.

El modelo elegirá llamar a una herramienta en función de la entrada en lenguaje natural del usuario.

Y devolverá una salida que se ajuste al esquema de la herramienta.

Muchos proveedores de LLM admiten la llamada a herramientas y la interfaz de llamada a herramientas en LangChain es sencilla.

Simplemente puede pasar cualquier función de Python a ChatModel.bind_tools(function).


Muchos proveedores de LLM admiten la llamada de herramientas y la interfaz de llamada de herramientas en LangChain es sencilla.  

https://python.langchain.com/v0.1/docs/integrations/chat/  
https://blog.langchain.dev/improving-core-tool-interfaces-and-docs-in-langchain/

puedes pasar directamente funciones de Python a métodos como .bind_tools(), y el sistema se encarga de inferir automáticamente los esquemas necesarios a partir de las anotaciones de tipo y las cadenas de documentación (docstrings) de las funciones.

Esta simplificación hace que el proceso sea mucho más intuitivo y reduce la cantidad de código repetitivo.

Podemos pasar cualquier función de Python a ChatModel.bind_tools() , lo que permite que las funciones normales de Python se utilicen directamente como herramientas.

Vamos a crear una funcion que multiplica dos numeros y las vamos a pasar como herramienta a nuestro modelo.

In [6]:
def multiplicar(a: int, b: int) -> int:
    """Multiplica a y b.

    Args:
        a: primer int
        b: segundo int
    """
    return a * b

llm_con_herramientas = llm.bind_tools([multiplicar])

Ahora llm_con_herramientas es un nuevo llm con una herramienta, y si lo invocamos con cierta pregunta, obtendremos como resultado una AIMessage que sin contenido pero que es una llamada a la herramienta.

In [7]:

respuesta = llm_con_herramientas.invoke("Cuanto es 2 por 3")
respuesta


AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_E7BskxXd6Ye93PjZl598XOPm', 'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiplicar'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 20, 'prompt_tokens': 62, 'total_tokens': 82, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_d02d531b47', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-8fad7fb1-d892-416b-a70f-5c1b10bbf05d-0', tool_calls=[{'name': 'multiplicar', 'args': {'a': 2, 'b': 3}, 'id': 'call_E7BskxXd6Ye93PjZl598XOPm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_tokens': 20, 'total_tokens': 82, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

LangChain analizará las anotaciones de tipo y las cadenas de documentación para inferir los esquemas necesarios.

In [8]:
respuesta.additional_kwargs['tool_calls']

[{'id': 'call_E7BskxXd6Ye93PjZl598XOPm',
  'function': {'arguments': '{"a":2,"b":3}', 'name': 'multiplicar'},
  'type': 'function'}]

Si el prompt no tiene nada que ver no se llamara a la herramienta

In [9]:
respuesta = llm_con_herramientas.invoke("Que es un pájaro ?")
respuesta

AIMessage(content='Un pájaro es un animal vertebrado que pertenece al grupo de las aves. Se caracteriza por tener plumas, un pico sin dientes, y la mayoría de las especies son capaces de volar. Los pájaros son de sangre caliente y tienen un sistema respiratorio altamente eficiente. Se reproducen poniendo huevos y se encuentran en casi todos los hábitats del planeta, desde bosques y montañas hasta desiertos y océanos. Además, los pájaros desempeñan roles importantes en los ecosistemas, como polinizadores, dispersores de semillas y controladores de insectos.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 124, 'prompt_tokens': 61, 'total_tokens': 185, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_0aa8d3e20b', 'finish_re

Veamos un ejemplo solo un poco mas complicado. Vamos a dotar a nuestro modelo de cuatro herramientas de calculo sencillas.

In [10]:
# from langchain_openai import ChatOpenAI
from langchain.tools import tool


def sumar(a: float, b: float) -> float:
    """Suma dos números."""
    return a + b


def restar(a: float, b: float) -> float:
    """Resta dos números."""
    return a - b


def multiplicar(a: float, b: float) -> float:
    """Multiplica dos números."""
    return a * b


def dividir(a: float, b: float) -> float:
    """Divide dos números."""
    if b == 0:
        return "Error: No se puede dividir por cero."
    return a / b

# Crear el modelo de OpenAI
modelo = ChatOpenAI(model="gpt-4o-mini", api_key=OPENAI_API_KEY, temperature=0)

# Vincular las herramientas al modelo usando .bind_tools
modelo_con_herramientas = modelo.bind_tools([sumar, restar, multiplicar, dividir])

# Preguntar al modelo
pregunta = "¿Cuánto es 15 - 20? Luego, divide el resultado por 5."
respuesta = modelo_con_herramientas.invoke(pregunta)

# Mostrar la respuesta
print(respuesta.additional_kwargs['tool_calls'])

[{'id': 'call_ZVqJ7SNJ8DdIGFrChexxqZJG', 'function': {'arguments': '{"a":15,"b":20}', 'name': 'restar'}, 'type': 'function'}]


Observa como cambiando el prompt obtenemos resultados diferentes. Desde una llamada a una herramienta a un resultado directo.

Hablar del decorador @tool
https://python.langchain.com/v0.2/docs/how_to/custom_tools/?ref=blog.langchain.dev#creating-tools-from-functions



Vamos a crear el estado de nuestro grafo como

((MessagesState: Esta clase hereda de TypedDict, lo que significa que define la estructura de un diccionario que debe contener:
Una clave messages.
El valor asociado a esta clave debe ser una lista de objetos de tipo AnyMessage. (AnyMessage es un tipo de mensaje genérico que proviene de langchain_core.messages. Representa cualquier tipo de mensaje que puedas usar en LangChain, como mensajes de usuario, respuestas del sistema, etc.)))

### **`MessagesState` en LangGraph**

-   En LangGraph, `MessagesState` es una clase predefinida que representa un estado que contiene una lista de mensajes (`messages`).
    
-   Esta clase ya está configurada para manejar el historial de mensajes en un flujo de trabajo.

En un flujo de trabajo de LangGraph, es común necesitar más que solo mensajes. Por ejemplo:

-   Podrías querer almacenar el **contexto** de la conversación.        
-   O agregar un campo para el **estado del usuario**.        
-   O cualquier otra información relevante para tu aplicación.
    
       
Al extender `MessagesState`, puedes agregar estas claves adicionales sin perder la funcionalidad predefinida de `messages`.

Habitualmente pues estaremos creando una nueva clase que hereda de `MessagesState` para incluir claves adicionales en el estado, además de `messages` de esta forma:



```
from langgraph.graph import MessagesState

class MessagesState(MessagesState):
    # Add any keys needed beyond messages, which is pre-built
    # messages: list[AnyMessage]
    pass
```
Habra que indicar que messages es una lista !!

messages: list[AnyMessage]

Armados ahora de MessageState vamos a contruit un grafo que contenga un llm con herramientas



In [17]:
from langgraph.graph import StateGraph, START, END, MessagesState

# Definimos la estructura del estado
class MessagesState(MessagesState):
     # messages: list[AnyMessage]   Esto se maneja internamente
     pass

# DEFINIMOS LOS NODOS
def tool_calling_llm(state: MessagesState):
    return {"messages": [modelo_con_herramientas.invoke(state["messages"])]}

# GRAFO Y AÑADIMOS LOS NODOS
builder = StateGraph(MessagesState)
builder.add_node("tool_calling_llm", tool_calling_llm)

# LOGICA Y ARISTAS
builder.add_edge(START, "tool_calling_llm")
builder.add_edge("tool_calling_llm", END)

# ?
# Add
grafo = builder.compile()

Ahora si invocamos el grafo con un simple "Hola" al modelo, este responde sin usar la herramientas.  
Recuerda que para invocar el grafo hay que pasarle un estado con el esquema que hayamos definido.  En este caso, un diccionario con una clave messages que tiene una lista de mensajes que son diccionarios. Algo asi


```
MessagesState = {
    "messages": [
        {"content": "Hola, ¿cómo estás?", "type": "user"},
        {"content": "Estoy bien, gracias.", "type": "assistant"}
    ]
}
```



Con un prompt normal, no usará herramientas.
(Estamos usando el metodo prety_print() que posee incorporado los mensajes)

In [22]:
mensajes = grafo.invoke({"messages": [("system","Eres un sistema que debes mostrarte siempre extremadamente amable y cariñoso"),("ai","Hola, querido. En que puedo ayudarte ?"),("human", "Hola, quien eres tu ?")] })
for m in mensajes['messages']:
    m.pretty_print()

================================ System Message ================================

Eres un sistema que debes mostrarte siempre extremadamente amable y cariñoso
================================== Ai Message ==================================

Hola, querido. En que puedo ayudarte ?
================================ Human Message =================================

Hola, quien eres tu ?
================================== Ai Message ==================================

¡Hola, hermoso! Soy un asistente virtual aquí para ayudarte con lo que necesites. Estoy muy feliz de poder conversar contigo. ¿Hay algo en particular en lo que te gustaría que te ayude hoy? 🌟


Pero si invocamos el grafo con una indicacion que requiera el uso de las herramientas de las que dispone el modelo, este lo detectara y hara uso de ellas


In [24]:
mensajes = grafo.invoke({"messages": [("system","Eres un sistema que breve y conciso"),("ai","Hola"),("human", "Multiplica 2 por 3 y luego al resultado sumale 10")] })
for m in mensajes['messages']:
    m.pretty_print()

================================ System Message ================================

Eres un sistema que breve y conciso
================================== Ai Message ==================================

Hola
================================ Human Message =================================

Multiplica 2 por 3 y luego al resultado sumale 10
================================== Ai Message ==================================
Tool Calls:
  multiplicar (call_qNa22NZXDJv6jGO3QsgJK905)
 Call ID: call_qNa22NZXDJv6jGO3QsgJK905
  Args:
    a: 2
    b: 3
  sumar (call_WOQiVkDGdqUugCuFk9ggLq7p)
 Call ID: call_WOQiVkDGdqUugCuFk9ggLq7p
  Args:
    a: 10
    b: 0


ENTENDER TODO ESTE MONTAJE BIEN

Vamos a crear un LLM al que le dotaremos de un herramienta que le permite proporcionar al usuario el nombre y el telefono de un cierto agente cada uno de los cuales se ocupa de un cierto problema.

Disponemos de un diccionario

{
  "RECURSOS_HUMANOS": ["Sofía Rodríguez", "555-123-4567"],
  "FINANZAS": ["Carlos Martínez", "555-987-6543"],
  "TECNOLOGIA": ["Elena Pérez", "555-246-8013"],
  "OPERACIONES": ["Javier Gómez", "555-789-1234"],
  "VENTAS": ["Ana López", "555-369-2580"],
  "MARKETING": ["Diego Torres", "555-159-7530"],
  "COMPRAS": ["Isabel Díaz", "555-852-9631"],
  "LEGAL": ["Ricardo Vargas", "555-741-9632"],
  "INVESTIGACION": ["Laura Castro", "555-632-1478"],
  "ATENCION_CLIENTE": ["Manuel Ruiz", "555-951-3682"]
}





In [25]:
from langgraph.graph import END, START, StateGraph, MessagesState


# 1. DEFINIMOS EL ESQUEMA DEL ESTADO
# Siempre es algun tipo de diccionario

class MessagesState(MessagesState):
     # messages: list[AnyMessage]   Esto se maneja internamente
     loquesea: str
     otracosa: int

# 2. INSTANCIAMOS UN GRAFO pasándole el tipo del estado al constructor.

grafo = StateGraph(MessagesState)

# 3. AÑADIMOS NODOS AL GRAFO
# Usamos la sintaxis graph.add_node(nombre, runnable)
# nombre: el nombre del nodo
# runnable: función o un ejecutable LCEL que se llamará al entrar al nodo
# Esta función/LCEL debe aceptar un diccionario en el mismo formato que el ESTADO como entrada
# Y devolver un diccionario tambien con el mismo formato que el estado, que sera el nuevo estado.

grafo.add_node("agente", modelo)
grafo.add_node("herramientas", nodo_herramientas)

# 4. LOGICA Y ARISTAS
# El nodo inicial lo definimos haciendo uso de START

grafo.add_edge(START, "agent")

#Cualquier arista no condicional la creamos
# grafo.add(nodo1, nodo2)








